# FAISS (Facebook AI Similarity Search) 가이드

## 개요

FAISS(Facebook AI Similarity Search)는 고밀도 벡터의 효율적인 유사도 검색과 클러스터링을 위한 라이브러리입니다. 메모리 크기를 초과하는 대용량 벡터 집합에서도 고속 검색이 가능하며, 매개변수 튜닝과 성능 평가를 위한 지원 도구를 제공합니다.

## 학습 목표

이 튜토리얼에서는 다음 내용을 다룹니다:

- FAISS 벡터 저장소의 핵심 개념과 구현 방법
- VectorStore 생성, 관리, 활용 전략
- 유사도 검색과 메타데이터 필터링 기법
- 벡터 저장소의 저장, 로드, 병합 등 운영 기능

## 주요 학습 구성

### 기본 구성
1. **환경 설정 및 데이터 준비**
2. **VectorStore 생성 방법**
   - from_documents 방법
   - from_texts 방법
   - 직접 초기화 방법

### 검색 및 관리
3. **유사도 검색 기법**
   - 기본 유사도 검색
   - 매개변수 조정 및 필터링
4. **문서 관리 기능**
   - 문서 추가 (add_documents, add_texts)
   - 문서 삭제
   - 벡터 저장소 병합

### 고급 활용
5. **저장 및 로드 기능**
   - 로컬 저장소 관리
   - 데이터 지속성 보장
6. **Retriever 인터페이스**
   - 검색 알고리즘 옵션
   - RAG 시스템 통합

### 기술적 특징

| 특징 | 설명 |
|------|------|
| **고성능 검색** | 수백만 개 벡터에서도 밀리초 단위 검색 |
| **메모리 효율성** | RAM 크기를 초과하는 데이터셋 처리 |
| **다양한 인덱스** | L2, Inner Product 등 다양한 거리 메트릭 |
| **확장성** | 분산 환경에서의 병렬 처리 지원 |

## 참고 자료

- [LangChain FAISS 문서](https://python.langchain.com/v0.2/docs/integrations/vectorstores/faiss/)
- [FAISS 공식 문서](https://faiss.ai/)

---

In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv(override=True)

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LangChain-Tutorial")

## 환경 설정

FAISS 벡터 저장소 실습을 위한 라이브러리 설정과 샘플 데이터셋을 준비합니다.

### 샘플 데이터셋

튜토리얼에서는 두 가지 도메인의 텍스트 파일을 사용합니다:
- **NLP 키워드 파일**: 자연어 처리 관련 용어
- **금융 키워드 파일**: 금융 분야 전문 용어

이러한 다중 도메인 데이터를 통해 메타데이터 필터링과 도메인별 검색 기능을 실습할 수 있습니다.

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 텍스트 분할 설정 (청크 크기: 600, 중복 없음)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=0)

# 텍스트 파일을 load -> List[Document] 형태로 변환
loader1 = TextLoader("data/nlp-keywords.txt")  # NLP 키워드 파일 로더
loader2 = TextLoader("data/finance-keywords.txt")  # 금융 키워드 파일 로더

# 문서 분할 - 로드와 동시에 설정된 크기로 분할
split_doc1 = loader1.load_and_split(text_splitter)
split_doc2 = loader2.load_and_split(text_splitter)

# 문서 개수 확인
len(split_doc1), len(split_doc2)

## VectorStore 생성

FAISS를 사용한 벡터 저장소 생성 방법을 알아봅니다. 각 방법은 사용 상황에 따라 적절히 선택할 수 있습니다.

### FAISS 초기화 매개변수

FAISS 벡터 저장소를 직접 생성할 때 사용하는 핵심 구성 요소입니다.

#### 주요 매개변수

| 매개변수 | 타입 | 설명 |
|----------|------|------|
| **embedding_function** | Embeddings | 텍스트를 벡터로 변환하는 임베딩 함수 |
| **index** | faiss.Index | 벡터 검색을 위한 FAISS 인덱스 객체 |
| **docstore** | Docstore | 원본 문서를 저장하는 문서 저장소 |
| **index_to_docstore_id** | Dict[int, str] | 인덱스 번호와 문서 ID 연결 매핑 |

### FAISS 핵심 개념

**FAISS(Facebook AI Similarity Search)**는 Meta에서 개발한 고성능 벡터 검색 라이브러리입니다.

#### 주요 특징

- **벡터 검색**: 텍스트를 벡터로 변환 후 유사한 벡터를 빠르게 검색
- **클러스터링**: 유사한 벡터들을 그룹으로 묶어 효율적 관리
- **대용량 처리**: 수백만 개의 벡터에서도 높은 성능 유지

#### 인덱스 유형

| 인덱스 타입 | 특징 | 적용 상황 |
|------------|------|----------|
| **IndexFlatL2** | L2 거리 기반 완전 검색 | 소규모 데이터, 높은 정확도 필요 |
| **IndexFlatIP** | 내적 기반 검색 | 코사인 유사도 검색 |
| **IndexIVFFlat** | 클러스터링 기반 근사 검색 | 대용량 데이터, 속도 우선 |

In [ ]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_openai import OpenAIEmbeddings

# OpenAI 임베딩 모델 설정 (text-embedding-3-small 사용)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 임베딩 차원 크기를 계산 (벡터 차원 확인)
dimension_size = len(embeddings.embed_query("hello world"))
print(dimension_size)

In [ ]:
# FAISS 벡터 저장소 생성
db = FAISS(
    embedding_function=embeddings,  # 임베딩 함수 지정
    index=faiss.IndexFlatL2(dimension_size),  # L2 거리 기반 인덱스 생성
    docstore=InMemoryDocstore(),  # 메모리 내 문서 저장소
    index_to_docstore_id={},  # 인덱스-문서 ID 매핑 (빈 딕셔너리로 시작)
)

### FAISS 벡터 저장소 생성 (from_documents)

`from_documents` 클래스 메서드는 Document 객체 리스트를 벡터 저장소로 변환하는 가장 직관적인 방법입니다.

#### 매개변수

| 매개변수 | 타입 | 설명 |
|----------|------|------|
| **documents** | List[Document] | 벡터 저장소에 추가할 Document 객체 리스트 |
| **embedding** | Embeddings | 텍스트를 벡터로 변환할 임베딩 함수 |
| **kwargs** | Dict | 추가 키워드 인자 |

#### 처리 과정

1. **내용 분리**: 각 Document의 `page_content`(텍스트)와 `metadata`(메타정보) 추출
2. **자동 변환**: 내부적으로 `from_texts` 메서드 호출
3. **저장소 생성**: 완성된 FAISS 벡터 저장소 반환

#### 사용 권장 상황

- 문서 로더를 통해 파일에서 Document 객체를 생성한 경우
- 메타데이터가 포함된 구조화된 문서를 처리할 때
- 기존 Document 객체를 벡터 저장소로 변환하고자 할 때

In [ ]:
# from_documents로 FAISS 벡터 저장소 생성
db = FAISS.from_documents(
    documents=split_doc1,  # 분할된 NLP 문서들
    embedding=OpenAIEmbeddings(model="text-embedding-3-small")  # 임베딩 모델 설정
)

In [ ]:
# 문서 저장소의 인덱스-ID 매핑 확인
db.index_to_docstore_id

In [ ]:
# 저장된 문서의 ID와 Document 객체 매핑 확인
db.docstore._dict

### FAISS 벡터 저장소 생성 (from_texts)

`from_texts` 클래스 메서드는 텍스트 리스트를 직접 벡터 저장소로 변환하는 방법입니다.

#### 매개변수

| 매개변수 | 타입 | 설명 |
|----------|------|------|
| **texts** | List[str] | 벡터 저장소에 추가할 텍스트 문자열 리스트 |
| **embedding** | Embeddings | 텍스트를 벡터로 변환할 임베딩 함수 |
| **metadatas** | Optional[List[dict]] | 각 텍스트에 대응하는 메타데이터 리스트 |
| **ids** | Optional[List[str]] | 각 텍스트의 고유 식별자 리스트 |
| **kwargs** | Dict | 추가 키워드 인자 |

#### 처리 과정

1. **벡터 변환**: 제공된 임베딩 함수로 모든 텍스트를 벡터로 변환
2. **인덱스 구축**: 변환된 벡터들로 FAISS 인덱스와 문서 저장소 생성
3. **저장소 완성**: 검색 가능한 벡터 저장소 반환

#### 사용 권장 상황

- 프로그래밍적으로 생성된 텍스트 데이터를 처리할 때
- 빠른 프로토타이핑이 필요한 경우
- 단순한 텍스트 문자열을 직접 벡터화하고자 할 때

#### 구현 고려사항

- **ID 관리**: 향후 문서 관리를 위해 의미있는 ID 지정 권장
- **데이터 일관성**: texts, metadatas, ids 리스트의 길이는 동일해야 함
- **메모리 효율성**: 대량 텍스트 처리 시 메모리 사용량 모니터링 필요

In [ ]:
# from_texts로 문자열 리스트에서 직접 생성
db2 = FAISS.from_texts(
    ["안녕하세요. 정말 반갑습니다.", "제 이름은 테디입니다."],  # 텍스트 리스트
    embedding=OpenAIEmbeddings(model="text-embedding-3-small"),  # 임베딩 모델
    metadatas=[{"source": "텍스트문서"}, {"source": "텍스트문서"}],  # 메타데이터 리스트
    ids=["doc1", "doc2"],  # 고유 ID 지정
)

### 저장 결과 확인

`from_texts`로 생성한 벡터 저장소의 내부 구조를 확인합니다. 지정한 ID 값들과 메타데이터가 올바르게 저장되었는지 검증할 수 있습니다.

In [ ]:
# 저장된 문서 내용과 ID 확인
db2.docstore._dict

## 유사도 검색 (Similarity Search)

`similarity_search` 메서드는 주어진 쿼리와 가장 유사한 문서들을 찾는 벡터 저장소의 핵심 기능입니다.

### 매개변수

| 매개변수 | 타입 | 기본값 | 설명 |
|----------|------|--------|------|
| **query** | str | - | 유사한 문서를 찾기 위한 검색 쿼리 텍스트 |
| **k** | int | 4 | 반환할 문서 개수 |
| **filter** | Optional[dict] | None | 메타데이터 기반 필터링 조건 |
| **fetch_k** | int | 20 | 필터링 전에 가져올 문서 수 |

### 검색 처리 과정

1. **쿼리 임베딩**: 입력된 쿼리 텍스트를 벡터로 변환
2. **유사도 계산**: FAISS 인덱스에서 가장 유사한 벡터들 검색
3. **결과 정렬**: 유사도가 높은 순서대로 문서 정렬
4. **문서 반환**: 상위 k개 문서를 Document 객체로 반환

### 주요 특징

| 특징 | 설명 |
|------|------|
| **고속 검색** | FAISS 최적화 알고리즘으로 대용량 데이터에서도 빠른 검색 |
| **유연한 설정** | k 값과 필터로 검색 범위와 결과 수 조절 |
| **메타데이터 필터링** | 특정 조건에 맞는 문서만 대상으로 검색 |
| **확장성** | fetch_k로 검색 범위와 성능 최적화 |

### 성능 최적화 가이드

#### 검색 속도 최적화
- **적절한 k 값**: 필요 이상의 많은 문서 요청 시 성능 저하
- **효율적 필터링**: 메타데이터 필터로 검색 범위 사전 제한
- **fetch_k 조정**: 필터 사용 시 충분한 후보 문서 확보

#### 검색 정확도 향상
- **쿼리 최적화**: 명확하고 구체적인 검색어 사용
- **도메인 특화**: 메타데이터 필터로 관련 도메인만 검색
- **임베딩 모델**: 작업에 적합한 임베딩 모델 선택

In [ ]:
# 기본 유사도 검색 (기본적으로 4개 문서 반환)
db.similarity_search("TF IDF 에 대하여 알려줘")

### 검색 결과 개수 조절

`k` 매개변수를 사용하여 반환받을 문서의 개수를 직접 지정할 수 있습니다. 작업 요구사항에 따라 적절한 결과 수를 설정하는 것이 중요합니다.

In [ ]:
# k=2로 설정하여 상위 2개 문서만 반환
db.similarity_search("TF IDF 에 대하여 알려줘", k=2)

### 메타데이터 필터링

`filter` 매개변수를 사용하여 메타데이터 정보를 기반으로 특정 조건에 맞는 문서만 검색할 수 있습니다. 이는 대용량 데이터에서 정확한 정보를 찾을 때 매우 유용한 기능입니다.

In [ ]:
# 메타데이터 필터를 적용한 검색 (NLP 키워드 파일에서만 검색)
db.similarity_search(
    "TF IDF 에 대하여 알려줘", 
    filter={"source": "data/nlp-keywords.txt"},  # 특정 소스로 필터링
    k=2  # 상위 2개 결과만 반환
)

### 도메인별 검색

동일한 쿼리라도 다른 `source` 필터를 적용하면 완전히 다른 결과를 얻을 수 있습니다. 이를 통해 특정 도메인이나 카테고리에 특화된 검색이 가능합니다.

In [ ]:
# 다른 소스로 필터링된 검색 (금융 키워드 파일에서만 검색)
db.similarity_search(
    "TF IDF 에 대하여 알려줘", 
    filter={"source": "data/finance-keywords.txt"},  # 금융 키워드 파일로 필터링
    k=2
)

## 문서 추가 및 관리

이미 생성된 벡터 저장소에 새로운 문서나 텍스트를 추가할 수 있습니다. 이는 실시간으로 데이터가 업데이트되는 시스템에서 매우 중요한 기능입니다.

### add_documents 메서드

`add_documents` 메서드는 기존 벡터 저장소에 Document 객체들을 추가하는 기능을 제공합니다.

#### 매개변수

| 매개변수 | 타입 | 설명 |
|----------|------|------|
| **documents** | List[Document] | 추가할 Document 객체들의 리스트 |
| **kwargs** | Dict | 추가 키워드 인자 (예: ids 지정) |

#### 처리 과정

1. **내용 추출**: Document의 `page_content`와 `metadata` 분리
2. **자동 변환**: 내부적으로 `add_texts` 메서드 호출
3. **ID 반환**: 추가된 문서들의 고유 ID 리스트 반환

#### 주요 특징

- **데이터 보존**: 기존 벡터 저장소의 데이터는 그대로 유지
- **자동 ID 생성**: ID를 지정하지 않으면 고유 ID 자동 생성
- **메타데이터 지원**: Document의 메타데이터가 완전히 보존

In [ ]:
from langchain_core.documents import Document

# Document 객체로 새 문서 추가
db.add_documents(
    [
        Document(
            page_content="안녕하세요! 이번엔 도큐먼트를 새로 추가해 볼께요",  # 문서 내용
            metadata={"source": "mydata.txt"},  # 메타데이터 설정
        )
    ],
    ids=["new_doc1"],  # 고유 ID 지정
)

In [ ]:
# 새로 추가된 문서 검색 테스트
db.similarity_search("안녕하세요", k=1)

### add_texts 메서드

`add_texts` 메서드는 문자열 형태의 텍스트를 직접 벡터 저장소에 추가하는 기능을 제공합니다.

#### 매개변수

| 매개변수 | 타입 | 설명 |
|----------|------|------|
| **texts** | Iterable[str] | 추가할 텍스트 문자열들의 이터러블 |
| **metadatas** | Optional[List[dict]] | 각 텍스트에 연결할 메타데이터 |
| **ids** | Optional[List[str]] | 각 텍스트의 고유 식별자 |
| **kwargs** | Dict | 추가 키워드 인자 |

#### 처리 과정

1. **텍스트 처리**: 입력된 텍스트 이터러블을 리스트로 변환
2. **벡터 변환**: `_embed_documents` 메서드로 텍스트를 벡터로 변환
3. **저장소 추가**: `__add` 메서드를 통해 벡터 저장소에 실제 추가
4. **ID 반환**: 추가된 텍스트들의 ID 리스트 반환

#### 구현 고려사항

| 항목 | 권장사항 | 이유 |
|------|----------|------|
| **ID 지정** | 의미있는 ID 사용 | 향후 문서 삭제/업데이트 용이성 |
| **배치 처리** | 다수 텍스트 동시 추가 | 처리 효율성 향상 |
| **메타데이터 활용** | 필터링용 메타데이터 추가 | 검색 정확도 향상 |

In [ ]:
# 텍스트 리스트로 직접 문서 추가
db.add_texts(
    ["이번엔 텍스트 데이터를 추가합니다.", "추가한 2번째 텍스트 데이터 입니다."],  # 추가할 텍스트들
    metadatas=[{"source": "mydata.txt"}, {"source": "mydata.txt"}],  # 각 텍스트의 메타데이터
    ids=["new_doc2", "new_doc3"],  # 각 텍스트의 고유 ID
)

In [ ]:
# 추가된 문서들의 인덱스-ID 매핑 확인
db.index_to_docstore_id

### 문서 삭제 (delete)

`delete` 메서드는 벡터 저장소에서 특정 ID의 문서를 완전히 제거하는 기능을 제공합니다.

#### 매개변수

| 매개변수 | 타입 | 설명 |
|----------|------|------|
| **ids** | Optional[List[str]] | 삭제할 문서들의 ID 리스트 |
| **kwargs** | Dict | 추가 키워드 인자 (현재 미사용) |

#### 삭제 처리 과정

1. **유효성 검사**: 입력된 ID들의 존재 여부 확인
2. **인덱스 매핑**: 삭제할 ID에 해당하는 FAISS 인덱스 번호 탐색
3. **FAISS 제거**: FAISS 인덱스에서 해당 벡터 완전 제거
4. **문서 삭제**: 문서 저장소에서 원본 문서 삭제
5. **매핑 업데이트**: 인덱스-ID 매핑 테이블 재구성

#### 중요 주의사항

| 주의사항 | 설명 |
|----------|------|
| **비가역성** | 삭제된 데이터는 복구 불가능 |
| **동시성 문제** | 멀티스레드 환경에서 삭제 충돌 가능성 |
| **데이터 무결성** | 삭제 후 반드시 결과 검증 필요 |
| **성능 영향** | 대량 삭제 시 인덱스 재구성으로 인한 지연 |

#### 사용 패턴

일반적으로 다음과 같은 상황에서 사용됩니다:
- **테스트 데이터 정리**: 개발/테스트 후 임시 데이터 제거
- **오래된 데이터 삭제**: 주기적인 데이터 클리닝
- **잘못된 데이터 수정**: 오류 데이터 삭제 후 재추가

In [ ]:
# 삭제 테스트를 위한 임시 데이터 추가
ids = db.add_texts(
    ["삭제용 데이터를 추가합니다.", "2번째 삭제용 데이터입니다."],  # 삭제할 텍스트들
    metadatas=[{"source": "mydata.txt"}, {"source": "mydata.txt"}],  # 메타데이터
    ids=["delete_doc1", "delete_doc2"],  # 삭제용 ID들
)

In [ ]:
# 추가된 ID들 확인 (삭제에 사용될 ID들)
print(ids)

### ID 기반 문서 삭제 실행

`delete` 메서드에 삭제할 문서의 ID 리스트를 전달하여 해당 문서들을 벡터 저장소에서 완전히 제거할 수 있습니다.

In [ ]:
# 지정된 ID들로 문서 삭제 실행
db.delete(ids)

In [ ]:
# 삭제 후 인덱스-ID 매핑 상태 확인 (삭제된 ID들이 제거됨)
db.index_to_docstore_id

## 저장 및 로드

벡터 저장소를 로컬 디스크에 저장하고 다시 불러오는 기능은 실제 운영 환경에서 매우 중요합니다. 한 번 구축한 벡터 저장소를 재사용할 수 있어 시간과 비용을 크게 절약할 수 있습니다.

### 로컬 저장 (save_local)

`save_local` 메서드는 FAISS 인덱스, 문서 저장소, 인덱스-문서 ID 매핑을 모두 로컬 디스크에 저장합니다.

#### 매개변수

| 매개변수 | 타입 | 기본값 | 설명 |
|----------|------|--------|------|
| **folder_path** | str | - | 저장할 폴더 경로 |
| **index_name** | str | "index" | 저장할 인덱스 파일 이름 |

#### 저장 처리 과정

1. **폴더 생성**: 지정된 폴더 경로가 없으면 자동으로 생성
2. **인덱스 저장**: FAISS 인덱스를 바이너리 파일로 저장
3. **메타데이터 저장**: 문서 저장소와 ID 매핑을 pickle 형식으로 저장

#### 활용 시나리오

| 시나리오 | 이점 |
|----------|------|
| **재사용** | 시간 소요가 큰 임베딩 작업을 한 번만 수행 |
| **배포** | 프로덕션 환경에 미리 구축된 벡터 저장소 배포 |
| **백업** | 중요한 벡터 데이터베이스의 백업 생성 |
| **오프라인 작업** | 네트워크 없이도 벡터 검색 가능 |

#### 보안 고려사항

- **접근 권한**: 저장 경로에 대한 적절한 쓰기 권한 필요
- **저장 공간**: 대용량 데이터의 경우 충분한 디스크 공간 확보
- **데이터 보안**: pickle 파일의 보안 위험성 고려 필요

In [ ]:
# FAISS 벡터 저장소를 로컬 디스크에 저장
db.save_local(
    folder_path="faiss_db",  # 저장할 폴더명
    index_name="faiss_index"  # 인덱스 파일명 지정
)

### 로컬에서 불러오기 (load_local)

`load_local` 클래스 메서드는 로컬 디스크에 저장된 벡터 저장소를 다시 메모리로 불러오는 기능을 제공합니다.

#### 매개변수

| 매개변수 | 타입 | 기본값 | 설명 |
|----------|------|--------|------|
| **folder_path** | str | - | 저장된 파일들이 있는 폴더 경로 |
| **embeddings** | Embeddings | - | 새로운 쿼리 처리용 임베딩 객체 |
| **index_name** | str | "index" | 불러올 인덱스 파일 이름 |
| **allow_dangerous_deserialization** | bool | False | pickle 파일 역직렬화 허용 여부 |

#### 로드 처리 과정

1. **보안 확인**: pickle 파일 역직렬화의 위험성을 확인하고 사용자 명시적 허가 요구
2. **인덱스 로드**: 저장된 FAISS 인덱스 파일을 메모리로 불러오기
3. **데이터 복원**: pickle로 저장된 문서 저장소와 ID 매핑 정보 복원
4. **객체 생성**: 불러온 데이터로 완전한 FAISS 객체 재구성

#### 핵심 요구사항

| 요구사항 | 설명 | 중요도 |
|----------|------|--------|
| **임베딩 일치** | 저장 시와 동일한 임베딩 모델 사용 | 필수 |
| **보안 허가** | `allow_dangerous_deserialization=True` 설정 | 필수 |
| **버전 호환성** | FAISS 버전 간 호환성 확인 | 권장 |

#### 보안 주의사항

pickle 파일은 임의 코드 실행이 가능하여 보안상 위험할 수 있습니다. **신뢰할 수 있는 소스에서 생성된 파일만** 로드하시기 바랍니다.

In [ ]:
# 저장된 FAISS 벡터 저장소를 다시 로드
loaded_db = FAISS.load_local(
    folder_path="faiss_db",  # 저장된 폴더명
    index_name="faiss_index",  # 로드할 인덱스 파일명
    embeddings=embeddings,  # 동일한 임베딩 객체 사용 (중요!)
    allow_dangerous_deserialization=True,  # pickle 역직렬화 허용
)

In [ ]:
# 로드된 벡터 저장소의 데이터 구조 확인
loaded_db.index_to_docstore_id

### FAISS 객체 병합 (merge_from)

`merge_from` 메서드는 두 개의 벡터 저장소를 하나로 합치는 강력한 기능입니다. 여러 소스의 데이터를 통합할 때 매우 유용합니다.

#### 매개변수

| 매개변수 | 타입 | 설명 |
|----------|------|------|
| **target** | FAISS | 현재 객체에 병합할 대상 FAISS 객체 |

#### 병합 처리 과정

1. **호환성 검사**: 두 벡터 저장소의 문서 저장소가 병합 가능한지 확인
2. **인덱스 조정**: 기존 인덱스 길이를 기준으로 새 문서들의 인덱스 번호 설정
3. **FAISS 병합**: 두 FAISS 인덱스를 물리적으로 병합
4. **문서 통합**: 대상 객체의 모든 문서와 메타데이터를 현재 저장소에 추가
5. **매핑 업데이트**: 인덱스-문서 ID 매핑 테이블을 통합된 상태로 갱신

#### 주요 특징

| 특징 | 설명 |
|------|------|
| **완전한 통합** | 인덱스, 문서, 메타데이터가 모두 통합됨 |
| **순차적 확장** | 기존 데이터를 보존하면서 새 데이터 추가 |
| **인덱스 연속성** | 인덱스 번호의 연속성을 자동으로 유지 |

#### 활용 사례

- **도메인 통합**: NLP, 금융, 의료 등 다양한 분야 문서를 하나로 통합
- **점진적 확장**: 시간이 지나면서 새로운 문서 컬렉션을 기존 저장소에 추가
- **분산 처리**: 여러 환경에서 구축한 벡터 저장소들을 최종 통합

#### 주의사항

| 항목 | 주의점 |
|------|--------|
| **구조 호환성** | 병합하는 두 객체가 호환 가능한 구조여야 함 |
| **ID 중복** | ID 중복에 대한 별도 검사가 없으므로 사전 확인 필요 |
| **원자성** | 병합 중 오류 발생 시 부분적으로 병합된 상태 가능 |
| **메모리 사용량** | 병합 과정에서 일시적으로 높은 메모리 사용량 |

In [ ]:
# 병합 테스트를 위해 저장된 db 다시 로드
db = FAISS.load_local(
    folder_path="faiss_db",
    index_name="faiss_index",
    embeddings=embeddings,  # 동일한 임베딩 모델 사용
    allow_dangerous_deserialization=True,
)

In [ ]:
# 병합할 두 번째 FAISS 벡터 저장소 생성 (금융 문서 사용)
db2 = FAISS.from_documents(
    documents=split_doc2,  # 금융 키워드 문서들 
    embedding=OpenAIEmbeddings(model="text-embedding-3-small")  # 동일한 임베딩 모델
)

In [ ]:
# 첫 번째 db의 인덱스-ID 매핑 상태 확인
db.index_to_docstore_id

In [ ]:
# 두 번째 db2의 인덱스-ID 매핑 상태 확인
db2.index_to_docstore_id

### 두 벡터 저장소 병합 실행

`merge_from` 메서드를 사용하여 두 개의 독립적인 FAISS 벡터 저장소를 하나로 통합합니다. 이를 통해 더 풍부하고 다양한 검색 결과를 얻을 수 있습니다.

In [ ]:
# 두 벡터 저장소 병합 (db에 db2를 병합)
db.merge_from(db2)

In [ ]:
# 병합 후 통합된 인덱스-ID 매핑 확인 (두 데이터베이스가 합쳐짐)
db.index_to_docstore_id

## Retriever 변환 (as_retriever)

`as_retriever` 메서드는 벡터 저장소를 LangChain의 표준 Retriever 인터페이스로 변환하는 기능입니다. 이를 통해 RAG (Retrieval-Augmented Generation) 시스템에서 활용할 수 있습니다.

### 매개변수

| 매개변수 | 타입 | 설명 |
|----------|------|------|
| **search_type** | Optional[str] | 검색 알고리즘 유형 |
| **search_kwargs** | Optional[Dict] | 검색 함수에 전달할 세부 설정 |
| ****kwargs** | Dict | 검색 함수에 전달할 추가 키워드 인자들 |

### 지원하는 검색 유형

#### 1. similarity (기본값)
- **용도**: 가장 일반적인 유사도 기반 검색
- **특징**: 빠르고 정확한 기본 검색 방식
- **권장 상황**: 일반적인 검색 요구사항

#### 2. mmr (Maximal Marginal Relevance)
- **용도**: 다양성을 고려한 검색 (중복 제거)
- **장점**: 유사한 문서들이 반복되지 않도록 다양성 확보
- **추가 매개변수**:
  - `fetch_k`: MMR 알고리즘에 전달할 문서 수
  - `lambda_mult`: 다양성 vs 관련성 균형 조절 (0~1)

#### 3. similarity_score_threshold
- **용도**: 임계값 기반 검색 (품질 보장)
- **장점**: 일정 유사도 이상의 문서만 반환
- **추가 매개변수**:
  - `score_threshold`: 최소 유사도 점수 임계값

### 공통 검색 매개변수

| 매개변수 | 타입 | 기본값 | 설명 |
|----------|------|--------|------|
| **k** | int | 4 | 반환할 문서 수 |
| **filter** | Optional[dict] | None | 메타데이터 기반 필터링 조건 |
| **fetch_k** | int | 20 | 필터링 전 가져올 문서 수 |

### RAG 시스템 활용

Retriever로 변환된 객체는 LangChain의 체인이나 에이전트에서 직접 사용할 수 있어, 다음과 같은 애플리케이션에 활용 가능합니다:

- **질문답변 시스템**: 문서 기반 정확한 답변 생성
- **문서 요약**: 관련 문서들을 종합한 요약본 제작
- **지식 기반 대화**: 외부 지식을 활용한 대화형 AI
- **정보 검색 시스템**: 효율적인 문서 검색 및 랭킹

### 기본 검색기 동작 확인

기본 설정으로 생성된 검색기는 4개의 문서를 유사도 순으로 반환합니다. 이는 가장 일반적으로 사용되는 설정입니다.

In [ ]:
# 검색기 테스트를 위한 통합 FAISS 벡터 저장소 생성
db = FAISS.from_documents(
    documents=split_doc1 + split_doc2,  # NLP와 금융 문서를 모두 합쳐서 생성
    embedding=OpenAIEmbeddings(model="text-embedding-3-small")
)

### 기본 검색기 설정

검색기의 기본 동작을 확인해봅시다. 별도 설정 없이 생성하면 유사도 기반으로 4개 문서를 반환합니다.

In [ ]:
# 기본 검색기로 변환 후 검색 수행
retriever = db.as_retriever()  # 기본 설정 (similarity, k=4)

# 검색 실행
retriever.invoke("Word2Vec 에 대하여 알려줘")

### MMR을 활용한 다양성 검색

MMR (Maximal Marginal Relevance) 알고리즘을 사용하면 유사하면서도 다양한 문서들을 가져올 수 있습니다.

#### MMR 매개변수 설정

| 매개변수 | 설명 | 권장값 |
|----------|------|--------|
| **k** | 최종 반환할 문서 수 | 4-6 |
| **fetch_k** | MMR 계산을 위해 먼저 가져올 문서 수 | k의 2-5배 |
| **lambda_mult** | 다양성 vs 관련성 균형 조절 (0~1) | 0.25-0.75 |

#### lambda_mult 값에 따른 효과

- **0에 가까울수록**: 더 다양한 문서 선택 (중복 최소화)
- **1에 가까울수록**: 더 관련성 높은 문서 선택 (정확도 우선)

In [ ]:
# MMR 검색 방식으로 다양성이 높은 문서 검색
retriever = db.as_retriever(
    search_type="mmr",  # MMR 알고리즘 사용
    search_kwargs={
        "k": 6,  # 최종 반환할 문서 수
        "lambda_mult": 0.25,  # 다양성 높게 설정 (0에 가까울수록 다양함)
        "fetch_k": 10  # MMR 계산을 위해 먼저 가져올 문서 수
    }
)
retriever.invoke("Word2Vec 에 대하여 알려줘")

### MMR 상위 문서만 선별

MMR 알고리즘을 통해 더 많은 후보 문서를 검토한 후 상위 2개만 선별하여 반환합니다. 이는 품질과 다양성을 동시에 확보하는 방법입니다.

In [ ]:
# MMR로 더 많은 후보에서 상위 2개만 선별
retriever = db.as_retriever(
    search_type="mmr", 
    search_kwargs={
        "k": 2,  # 최종 2개만 반환
        "fetch_k": 10  # 10개 후보에서 선별
    }
)
retriever.invoke("Word2Vec 에 대하여 알려줘")

### 임계값 기반 품질 보장 검색

특정 유사도 임계값 이상의 문서만 반환하여 검색 결과의 품질을 보장합니다. 낮은 품질의 문서는 자동으로 제외됩니다.

In [ ]:
# 임계값 기반 검색 - 유사도 0.8 이상인 문서만 반환
retriever = db.as_retriever(
    search_type="similarity_score_threshold", 
    search_kwargs={
        "score_threshold": 0.8  # 최소 유사도 점수 설정
    }
)

retriever.invoke("Word2Vec 에 대하여 알려줘")

### 최고 유사도 문서만 검색

가장 관련성이 높은 단일 문서만 반환하고 싶을 때 사용합니다. 정확한 답변이 필요한 상황에 적합합니다.

In [ ]:
# 가장 유사한 단일 문서만 반환 (k=1 설정)
retriever = db.as_retriever(
    search_kwargs={"k": 1}  # 최상위 1개 문서만 반환
)

retriever.invoke("Word2Vec 에 대하여 알려줘")

### 메타데이터 필터링 활용

특정 메타데이터 조건에 맞는 문서만 대상으로 검색을 수행합니다. 이를 통해 특정 도메인이나 카테고리에 특화된 정확한 검색이 가능합니다.

In [ ]:
# 메타데이터 필터를 적용한 검색기 생성
retriever = db.as_retriever(
    search_kwargs={
        "filter": {"source": "data/finance-keywords.txt"},  # 금융 키워드 문서만 대상
        "k": 2  # 상위 2개 문서 반환
    }
)
retriever.invoke("ESG 에 대하여 알려줘")